<img src="./img/CCIT_Logo.png" alt="CCit Logo" width=400>

# 05 - Correlation Power Analysis (DPA) 🕵️

## ⚠️ Prerequisites

Hol' up - before continuing, ensure you have done the following:

* ☑ Read the introduction to Side-Channel Analysis in the [02-Intro to SCA](./02-Intro%20to%20SCA.ipynb) notebook.
* ☑ Learn how Simple Power Analysis works in [03-Simple Power Analysis](./03-Simple%20Power%20Analysis.ipynb) notebook.
* ☑ Learn how Differential Power Analysis works in [04-Differential Power Analysis](./04-Differential%20Power%20Analysis.ipynb) notebook.

### 📑 Summary

*In this lab, you'll learn what is Correlation Power Analysis, AKA* **CPA**, *how to carry out such an attack and what are its main differences compared to DPA. We will learn what is the main rationale behind this attack and why this is the fastest and most efficient way to retrieve a secret key. Finally, we will recall why CW's synchronous sampling is so important for both DPA and CPA attacks.*

### 💬 Learning outcomes
* What is CPA
* How to carry out such an attack
* What is the rationale behind, what phenomena it exploits and why it's more effective than DPA
* Why synchronous sampling improves the effectiveness of both DPA and CPA attack
* What are the challenges a real hardware hacker must face in a real world scenario

## 📑 Table of Contents
0. [Our secret key!](#key)
1. [What is Correlation Power Analysis? ❓](#cpa)
2. [How does CPA work?](#how)
    1. [Recalling DPA's rationale](#recalling)
    2. [CPA's rationale](#rationale)
    3. [CPA's leakage model](#leakage_model)
    4. [Correlating the estimated leakage to the real power consumption](#correlating)
    5. [Overview of the attack implementation](#overview)
3. [[Optional] Why synchronous sampling is so important](#sync_samp)
4. [Additional - Implementation and plots](#additional)
5. [Conclusions](#conclusions)

#### Before starting, I suggest you running the following snippets of code. They will expand your workspace both horizontally and vertically, helping you to better visualize the plots we are going to create.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# 0) Our secret key! <a class="anchor" id="key"></a>
The snippet below defines the secret key we need to find, run it so you'll have it available for the entire notebook.

In [ ]:
secret_key = [0x2B, 0x7E, 0x15, 0x16, 0x28, 0xAE, 0xD2, 0xA6, 0xAB, 0xF7, 0x15, 0x88, 0x09, 0xCF, 0x4F, 0X3C]

# 1) What is Correlation Power Analysis? ❓ <a class="anchor" id="cpa"></a>
1. [Correlation Power Analysis (CPA)](https://en.wikipedia.org/wiki/Power_analysis#Differential_power_analysis)

Correlation Power Analysis is the second and last statistics-based algorithm we are going to see. Given it's similarity to Differential Power Analysis, both DPA and CPA are often considered as the same class of attacks; people often simply refer to them as "DPA".

In reality, CPA is the most effective of the two, being able to retrieve the key in a faster way, with less traces, and achieving better results. As before, collecting multiple power traces is mandatory, as the attack is unapplicable with only a single trace. Moreover, the rule "the more traces the better the result" is still valid.

# 2) How does CPA work? <a class="anchor" id="how"></a>
In this tutorial, I will not ask you to implement CPA from scratch as it may take some time (similarly to DPA). Of course, nobody's stopping you from doing it: implementing algorithms from scratch is an incredibly powerful way to learn how things work and get implemented.

## 2.1) Recalling DPA's rationale <a class="anchor" id="recalling"></a>
Let's quickly recap how DPA works, so to better highlight the differences with CPA later.

### How do we retrieve all the key bytes that make up the secret key?
DPA follows a "divide-et-impera" approach: the entire key is found by retrieving all the key bytes one by one, starting from the first one. Once the first byte is retrieved, we apply the exact same algorithm with the second byte, and so forth until all 16 bytes are evaluated.

CPA is no different in this, **what changes is the correlation algorithm applied to each key byte to be found.**

### How do we find the correct key byte?
When evalauting a single key byte, DPA adopts a "brute force" approach: the attack proceeds by enumerating all possible values a key byte can assume, from `0x00` to `0xFF`, for a totality of 256 values. For each of these values, DPA assigns a score to it.

At the end of the evaluation, the key guess having obtained the highest score among its peers gets elected as "correct key byte".

### What is this score and how it is computed?
The score indicates, for each key guess tested, "how close its value is to the real key byte". However, the way this score is computed is a little more "complex" than that.

Indeed, to find "how close the key guess value is to the real key byte" we need to leverage the following:
1. A *model / ideal replica* implementing (parts of) the steps executed by the targeted algorithm (i.e. the algorithm under attack)
2. A *prediction function*, applying a prediction/assumption to a value produced by the above model/replica. Such assumption, if held true, will allow us to discriminate the correct key guess from the wrong ones. In most of the cases, DPA leverages the "LSB-based" prediction function: with this policy, we assume that the value of a single bit (the LSB) of the intermediate byte (produced by the replica) reflects itself on the power traces we collected. This policy is applied by the *selection function*, see the next point.
3. A *selection function* applying the policy dictated by the *prediction function* to the power traces we collected. The *selection function* partitions the power traces into two sets, following the value assumed by the target bit: if the bit is `'1'` we assume this value is somewhat present (and visible) in the power trace, hence the trace goes in the corresponding set. On the other hand, if the bit is `'0'`, we assume this value is not present in the power trace: the trace is put aside in the second set.
4. Some computations, that calculate, for each of the two sets, an average "representative" trace. Once the two average traces obtained, a single "differential" trace is computed from them. Such trace, if everything was implemented correctly, shall highlight the presence of correlation spikes, indicating the effective presence of the target bit's own power contribute. The score assigned to each key guess is simply the value of the global (or local, if using windowing) maximum peak recorded on the differential trace. 

**The entire rationale works as "reductio ad absurdum/argumentum ad absurdum".** We suppose, a priori, that the target bit has some influence on the power traces we collected. The assumption makes sense, but how do we make sure this happens for real? We force the assumption, *faking 'til we make it*, and see if it helds true or not.

Once a result is obtained, we "backtrace" the assumption: 

1. if a certain key guess has the highest score among its peers, this means that:
2. the differential trace showed some peaks and, by construction, these peaks highlight the average differences among the traces we partitioned into the two sets. This means that:
3. a vast majority (if not all) traces got partitioned correctly in their respective sets: the traces having the contribute of the target bit were partitioned in set_1, all the others in set_0. This means that the *selection function* did good, therefore:
5. the policy of the prediction function, which is applied by the selection function, is correct. This means that:
6. the power traces reflect the value of the target bit (previusly, we assumed the opposite), therefore:
7. the intermediate value, from which the target bit is extrapolated, is correct most of the times (if not always). This means that:
8. if the value produced by the replica is correct (and the replica itself is correct, and the plaintext byte is related to the power trace produced), then the only thing "left" is the key guess, which proves to be correct as it's the only variable, by exclusion, which must be correct!! 

## 2.2) CPA's rationale <a class="anchor" id="rationale"></a>
CPA's main rationale improves the one defined by DPA. Instead of focusing on the value assumed by a single bit of the intermediate and its contribute to the power trace, CPA correlates the entire intermediate (i.e. all of its bits) to the power trace. **Simply put, CPA improves DPA's rationale by extending it to multiple bits.**

The main differences among DPA and CPA, therefore, relies on the policy defined by the *prediction function* and, consequently, on the *selection function* used to apply it. Apart from that, CPA copies the same "divide-et-impera" approach adopted by DPA, leverages the exact same replica etc.

> ⚠️ What we previously called *prediction function*, with DPA, now it is called *leakage model*, in the case of CPA. Similarly, the *selection function* is now referred to as *evaluation function*. We will refer to this new nomenclature from now on.


## 2.3) CPA's *leakage model* - Assumptions on the power leakage <a class="anchor" id="leakage_model"></a>
Let's recall the processor bus and the data transfers among the CPU core and the register file we mentioned in the previous tutorial. 

That example was done for a specific reason: more often than not, data transfers on PARALLEL buses compose a substantial percentage of the overall power consumption of a processor core. Why parallel? Because parallel buses can move multiple bits at a time, increasing data transfers throughput, at the expense of die area and power consumption. The reason is clear: you want your core to transfer (and process) data as fast as possible, and parallel buses are the best solution in case of such short distances (from tens to hundreds of nanometers).

Let's deep dive into a microprocessor core architecture for a moment, down to the transistor level, where MOSFETs and finFETs operate. From a point of view of the transistors driving these buses, each bus line is seen as a big capacitance to be charged and discharged whenever there's a new byte to be transferred. Charging and discharging a capacitance requires a certain amount of eletrical charges, i.e. of energy. Of course, the higher the physical length of the interconnect, the higher the capacitance to drive, hence the higher the energy required to transfer every bit. At the same time, **the higher the number of bits to be transferred, the higher the energy required and thus the power consumption.** 

Let's assume, to keep things simple, that all bus lines have the same capacitance $C_{line}$. Therefore, each toggling bit (i.e. each bus line moving from `'0'` to `'1'` or from `'1'` to `'0'`) requires a certain amount of minimum energy to be charged and discharged, let's call it $x$.

Consider the following scenarios:

1. CPU requests Byte `0x0F` from register `AX`, hence it transfers `0000|1111` from the register file to the ALU internal registers: to do so, the 4 lowest bus lines are all set to the logic value `'1'`. 
> The energy required is $4x$;
2. CPU requests Byte `0xFF` from register `BX`, hence it transfers `1111|1111` from the register file to the ALU internal registers: to do so, all the bus lines are set to the logic value `'1'`. 
> The energy required is $8x$;
2. CPU requests Byte `0x40` from register `CX`, hence it transfers `0100|0000` from the register file to the ALU internal registers: to do so, the second highest bus line is set to the logic value `'1'`.
> The energy required is $x$.

**As we can see, the power consumption increases linearly with the number of bits set @ `'1'`.**

This is the basic assumption made by CPA: **the power consumption relates linearly to the Hamming Weight of the intermediate value computed.**

> ℹ️ The Hamming Weight (HW) of a Byte is an integer number indicating the number of `'1'`s that Byte is composed of.

So, for instance:
* `0x03` has $HW = 2$ (because `0x03` = `0000|0011` and there's 2 bits set @ `'1'` in there)
* `0x43` has $HW = 3$ (because `0x43` = `0100|0011` and there's 3 bits set @ `'1'` in there)
* `0x23` has $HW = 3$ (because `0x23` = `0010|0011` and there's 3 bits set @ `'1'` in there)
* `0xF7` has $HW = 7$ (because `0xF7` = `1111|0111` and there's 7 bits set @ `'1'` in there)
* `0x00` has $HW = 0$ (because `0x00` = `0000|0000` and there's 0 bits set @ `'1'` in there)

Hence, according to CPA's rationale, the above Bytes can be sorted from the highest power consumption to the lowest:
* `0xF7` > `0x23` = `0x43` > `0x03` > `0x00`

### 2.3.1) Additional information:

##### 0) Notes for the attacker:
> ℹ️ CPA is most effective when the device leakage model is known by the attacker: if the attack leads to poor results the cause generally relates either to a low-leaky device implementation or to a poor assumption of the leakage model. Similarly to the *prediction function* in the case of DPA, the attacker can test multiple leakage models to assess which one leads to better results, trading off the additional flexibility to the time required to
end the attack.

##### 1) [Optional] in-depth comments:

> ⚠️ In reality, CPA, in the most generic case, uses the Hamming Distance (HD) leakage model instead of the Hamming Weight (HW) one. The Hamming Distance is the difference of the Hamming Weigths of two consecutive bytes transferred on the bus and can be computed as follows (where $i$ indicates the *i-th* data transfer):
>
> $HD = HW(Byte_{i-1}) \oplus HW(Byte_{i})$
>
> Basically, the Hamming Distance counts how many bits toggled between the current Byte transfer and the previous one.
> The case considering the Hamming Weight, which is an oversemplification of the Hamming Distance one, simply assumes the Hamming Distance to be always equal to the Hamming Weight of the *i-th* Byte transfered: 
>
> $HD = HW(Byte_{i})$
> 
> For that to be possible, we need  $HW(Byte_{i-1}) = 0$, hence $Byte_{i-1} = $`0x00`

##### 2) [Optional] 
> You can read more about these details [in the original CPA paper](https://www.iacr.org/archive/ches2004/31560016/31560016.pdf).

##### 3) **[HIGHLY SUGGESTED]** [YouTube] 
> [A really good presentation by Sean Newman on Correlation Power Analysis](https://www.youtube.com/watch?v=xBPp5B8AJkQ)


## 2.4) Correlating the estimated leakage to the real power consumption <a class="anchor" id="correlating"></a>
As with DPA, once we have an assumption on the power leakage, we need to verify its correctness against the real system. 

##### How do we correlate the new assumption rationale (i.e. the leakage model) with the power traces?

We use maths! And the word "correlation" has never been so appropriate!
When mentioning the Hamming Weight leakage model, it's important to put the focus on the word "linear":

> CPA assumes the power consumption of the device depends **linearly** to the Hamming Weight of the intermediate value

So, in order to prove the correctness of that assumption, we need a tool able to highlight the presence of a linear correlation among the Hamming Weight of your intermediate value and the samples that compose the power traces we collected. 

> ℹ️ In jargon, this tool is also called *evaluation function*.

##### What statistics tool is used to highlight the presence of a linear correlation among two populations?
This tool is called *Pearson's correlation coefficient* or *PCC*. Please, [read the related Wikipedia page](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient).

$$\rho_{X,Y} = \frac{cov(X,Y)}{\sigma_X \sigma_Y}$$

We can break the above equation into some smaller operations, for instance $cov(X, Y)$ is the covariance of the two datasets `X` and `Y` and can be calculated as follows:

$$cov(X, Y) = \sum_{n=1}^{N}[(Y_n - \bar{Y})(X_n - \bar{X})]$$

$\sigma_X$ and $\sigma_Y$ are the standard deviation of the two datasets. This value can be calculated with the following equation:

$$\sigma_X = \sqrt{\sum_{n=1}^{N}(X_n - \bar{X})^2}$$

> ℹ️ If the above equation scares you, don't worry, we don't need to understand every detail out of it. We simply need to apply it and understand the following:

$\rho_{X,Y}$ is the value of *Pearson's correlation coefficient* and describes how close the two variables $X$ and $Y$ are linearly related. 

By design, **it varies among $-1$ and $+1$**, this means that:

* If $Y$ always increases when $X$ increases, PCC's value will be $1$;
* If $Y$ always decreases when $X$ increases, PCC's value will be $-1$;
* If $Y$ is totally independent of $X$, PCC's value will be $0$.

So, the formula seems complex, but the result is quite easy to understand, right? In the following image, taken from [wikipedia](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), there's a visual representation demonstrating how Pearson's coefficient behaves:

Note that the correlation reflects the strength and direction of a linear relationship (top row), but not the slope of that relationship (middle), nor many aspects of nonlinear relationships (bottom).

<img id="pearson" src="./img/05-Correlation Power Analysis/pearson.svg" alt="Pearson's Correlation Coefficient" width=600>

By DenisBoigelot, original uploader was Imagecreator - Own work, original uploader was Imagecreator, CC0, https://commons.wikimedia.org/w/index.php?curid=15165296

## 2.5) Overview of the attack implementation <a class="anchor" id="overview"></a>
We won't implement from scratch the entire attack. We'll limit ourselves to highlight the main points of it.
### 2.5.1) Assumptions and preparatory operations
Similarly to what we did with DPA, let's simplify the attack by considering the following:
1. We collected 50 power traces, along with their respective 50 plaintext vectors. Each power traces consists of a list of 5000 samples, each plaintext is a list of 16 bytes:
```python
power_traces = [
    [0.23, 0.12, -0.45, ... .... ..... , 0.15],  # plaintext0
    .                                            # plaintext1
    .                                            # plaintext2
    .
    [1.52, 0.93, 0.43,  ... .... ..... , 0.14]   # plaintext49
]
plaintexts = [
    [0x34, 0xA4, 0xFB,  ... .... ..... , 0x29],  # trace0
    .                                            # trace1
    .                                            # trace2
    .
    [0x74, 0xF3, 0x55,  ... .... ..... , 0x21]   # trace49
]
```
2. We are going to attack only Byte \#0 of the key (it should be clear now how the attack retrieves the entire key)
3. We already built the same ideal replica we used with DPA:
```python
def aes_internal(pt_byte, key_guess) -> int:
    intermediate_byte = sbox[pt_byte ^ key_guess]
    return intermediate_byte
```
4. We already built a function able to implement the Hamming Weight leakage model, given the intermediate computed above:
```python
def HW(intermediate_byte) -> int:
    return bin(intermediate_value).count('1')
```

### 2.5.2) Attack implementation
The goal is to build a data structure similar to this one below:
```python
    data_struct = [ 
                      [ HW[aes_internal(key_guess, plaintext_0[0])],  trace_0[0]  ],
                      [ HW[aes_internal(key_guess, plaintext_1[0])],  trace_1[0]  ],
                      [ HW[aes_internal(key_guess, plaintext_2[0])],  trace_2[0]  ],
                      ...
                      ...
                      [ HW[aes_internal(key_guess, plaintext_49[0])], trace_49[0] ]
                  ]
```
Let's break it out. Each inner list inside the outer list contains two elements:
1. `HW[aes_internal(key_guess, plaintext_0[0])]`:
> the Hamming Weight (or supposed leakage) a specific intermediate is producing, computed using the current key guess together with the first byte (`[0]`) of the *i-th* plaintext (`plaintext_0`, `plaintext_1` etc...)
2. `trace_0[0]`:
> the first sample point (`[0]`) of the *i-th* power trace (`trace_0`, `trace_1` etc...)

And then use numpy to address it in a clever manner:
```python
import numpy as np
data_struct = np.asarray(data_struct)

# Build a column vector with ALL the first elements of the inner lists (the Hamming Weights)
X = data_struct[:,0]
print(X)
> [
    HW[aes_internal(key_guess, plaintext_0[0])],
    HW[aes_internal(key_guess, plaintext_1[0])],
    HW[aes_internal(key_guess, plaintext_2[0])],
    ...
    HW[aes_internal(key_guess, plaintext_49[0])],
  ]

# Build a column vector with ALL the second elements of the inner lists (the first sample point of all traces)
Y = data_struct[:,1]
print(Y)
> [
    trace_0[0],
    trace_1[0],
    trace_2[0],
    ...
    trace_49[0]
  ]
```

This data structure contains the datasets `X` and `Y` to be used to compute Pearson's Correlation Coefficient.
* `X = data_struct[:,0]`
* `Y = data_struct[:,1]`

Then, we can proceed by computing the PCC following its formula:

$$\rho_{X,Y} = \frac{cov(X,Y)}{\sigma_X \sigma_Y}$$

What we obtain is a float number indicating the presence (or not) of a linear correlation among the Hamming Weight of the intermediate and the power consumption observed **in all the traces, but only in the point in time corresponding to the first sample (out of the 5000 available)!**

> ⚠️ This means, similarly to DPA, that CPA has to "build" an entire trace, evaluating all the 5000 samples to highlight the point in time having the highest correlation value. Therefore, the above explanation and related computations must be repeated for all the 5000 samples that compose the traces. Basically, you have to compute a `data_struct` matrix for every of the 5000 samples collected.

Once you computed 5000 PCC values, you obtain a trace (a list) with all of them. Like DPA, you extract the highest absolute correlation value from it, and you use it as a score to be assigned to the current key guess under evaluation.

Finally, as with DPA, you build a list of 256 `(key_guess, score)` tuples, and you later extract the key guess having the highest PCC score: **this value is (with high chance) the value of the secret key byte!**

> ℹ️ The entire CPA attack is really complex and nested. Think about it: for every byte of the 16 that compose the secret key, you have to test 256 possible key guesses and for each of them you have to compute 5000 PCC values (one for each sample). However, most of the computations can be offloaded to numpy functions, simplifying massively some iterations and speeding up the code quite a bit. 

# 3) [Optional] Why synchronous sampling is so important <a class="anchor" id="sync_samp"></a>
In the notebook dedicated to Simple Power Analysis we mentioned, in section *2.5 [Optional] - We need to go deeper* how ChipWhisperer's sampling and capture operations work under the hood. I promised you we would get back to the topic when talking about statistical attacks, and here we are!

Synchronous sampling allows to massively reduce the number of traces needed to complete an attack, but how so? Well, take a look at the following image:

<img id="trace_alignment" src="./img/05-Correlation Power Analysis/trace_alignment.png" alt="Trace alignment" width=400>

With synchronous sampling you can have the following scenarios:
1. Both the target board and the capture one share the exact same clock signal
> So, for instance, if target clock runs @ 10MHz, also capture clock runs @ 10MHz. DPA and CPA may still be possible (I haven't tried actually), but (I believe) really hard to achieve with a low number of traces collected. To maximize your chances you'd want to have your capture clock to be **at least** twice as fast as the target clock (Nyquists's sampling theory).
2. The capture board receives the same clock signal as the target board, but with the frequency multiplied by a certain factor
> In general, the higher the multiplier among the two clocks, the finer the sampling granularity and the amount of information you can extract from a trace.
>
> ChipWhisperer, for instance, has a synchronous capture clock, which is also exactly 4 times faster than the target clock.
3. Same as 2. explained above, but the rising edges of the two clock have a slight **constant** delay among them
> Still good, can still be considered a form of synchronous sampling: the two clocks are slightly misplaced, but their frequency and phase relation is constant over time 

No matter how these 2 clock signals are generated, what's important is that **they must have a constant phase and frequency among them**: basically, they must not run freely, drifting away one from the other. If that happens, then our captured traces will no more have the same phase relationship, meaning that, for instance, a sample captured @ $80 \mu s$ from the capture start in the first trace may be captured @ $84 \mu s$ in the second trace, and @ $90 \mu s$ in the third trace. Basically, what happens can be seen in the first plot of the above image: all the traces have lost the time reference and are all scrambled up!

> After a certain number of traces we no more have a clear reference dictated by the samples! **Therefore, while DPA and CPA statistical computations still refer to the same sample, that sample does no more refer to the same point in time trace after trace! In this scenario, highlightning the correlation spikes becomes incredibly hard and the same attack requires an enourmous amount of power traces to achieve the same result as synchronous sampling!**

> **Asynchronous** sampling collects traces affected by **jitter**, meaning that, even if they refer to the same operation and got collected using the same trigger signal, their temporal misalignment reduces the quality and the efficacy of the statistical attack that leverages them.

##### How to solve problems introduced by asynchronous sampling?

1. As said above, we can massively increase the number of traces collected: DPA and CPA can still retrieve the key. However, a higher number of traces requires more memory to store all the samples collected and more time to complete the statistic computations (as more iterations are required). It is definitely not the ideal scenario.

2. We can re-align the traces captured before analyzing them with DPA and CPA. This is an extra step: time is required so as a little bit of additional expertise (the process can be automated though). In the image above, bottom graph, you can see how aligned traces look like.

# 4) Additional - Implementation and plots <a class="anchor" id="additional"></a>
If you want, try to implement your own CPA attack following the overview above. Leverage numpy and its documentation: many array/list manipulations become really simple with it. 

> To collect the power traces, you can refer again to the `firecracker` program codename we used in the previous DPA notebook.

Moreover, if you are feeling even braver, try to imitate the following plots with matplotlib. You can find all the links to documentation in the previous tutorials, or look for docs on the internet.



This graph shows, superposed, all the correlation traces referring to the correct bytes of the secret key: the purpose of this graph is to highlight how the correlation spikes move (in time) as the byte changes. The position of the peaks show you where each key byte is handled and the related intermediate value is computed! For instance, the zone among sample \#1000 and sample \#2000 refers to the first `SubBytes` step, just after the `AddRoundKey` one!

> ⚠️ The correlation traces produced in the plot were obtained using DPA on a different `AES-128` implementation: try to produce a similar graph with your own CPA implementation

<img id="dpa_aes_keys" src="./img/05-Correlation Power Analysis/dpa_aes_keys.png" alt="DPA correlation traces superposed" width=1200>

# 5) Conclusions <a class="anchor" id="conclusions"></a>
That's it for this tutorial!

Now, we have one tutorial left beore the Jeopardy CTFs challenges: **in the following notebook I will introduce you to some countermeasures used to prevent power side-channel analysis.**

---
Do you have suggestions, doubts or simply need to reach me?
* 🐦 Twitter: `[at]mrcuve0`
* 💬 Discord: `Mrcuve0#4179`
* 💻 GitHub:  `Mrcuve0`
* 📬 Email: `mrcuve0 [at] posteo [d.ot] net`
* 💼 LinkedIn:
> Please, send me a message via the previous methods before asking for my LinkedIn

---
Copyright ©️ [CINI - Cybersecurity National Lab](https://cybersecnatlab.it/) - Torino, 2022.

> This material is a derivative work of [NewAE's official Jupyter Notebooks](https://github.com/newaetech/chipwhisperer-jupyter), distributed under the open-source GPL license. You can distribute and modify this material (even for commercial trainings), provided you maintain references to this repository and the original authors, and also offer your derived material under the same conditions.


ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

All other product names, logos, and brands are property of their respective owners.

The software is provided 'as is', without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.